Daniel Aguilar
Homework3
CSP 228
Prof. James Liporace
04/14/2025

**Instructions**  
Progrmming Projects

Chapter 6 

6.2,6.3,6.4

Chapter 7 

7.1,.7.2,7.3,.7.4



**Problem 6.2**  

Every positive integer can be divided into a set of integer factors. The
minimum set of factors must be a collection of prime numbers, where a
prime number is one that is only evenly divisible by 1 and itself. Write a
recursive function, factor(), that returns the list of integer factors of x.

If you haven’t worked with factoring before, it is helpful to know that
only the factors between 2 and the square root of x need to be tested.
Instead of doing those with a loop, add a second, optional parameter to
factor(x, lowest) that is the lowest possible integer factor of x. The
recursive function should check for the base case(s), and make recursive
calls based on whether lowest evenly divides x (x % lowest == 0). If
it does, then add lowest to the factors of x divided by lowest. If lowest
doesn’t evenly divide x, then look for factors with the next higher
possible factor. Ideally, the next higher factor would always be the next
higher prime number. There are algorithms for that, but for this
exercise, it’s sufficient to try the next higher integer. You may use
Python’s built-in list structure to assemble the factors or the
LinkedList structure of Chapter 5. Note that a factor can appear more
than once in the final list. For extra utility, handle requests for the
factors of negative integers by returning the factors of the positive
version but with the factor 1 replaced by −1. Test your function on some
compound (nonprime) and prime numbers and the special cases of 0
and 1.